In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import auth
import pandas as pd

In [2]:
scope = "playlist-modify user-read-private user-read-recently-played user-library-read user-read-currently-playing"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=auth.CLIENT_ID, client_secret=auth.CLIENT_SECRET, redirect_uri=auth.REDIRECT_URI))

In [3]:
def get_all_current_user_saved_tracks():
    offset = 0
    saved_tracks_group = []

    while(True):
        
        saved_tracks = sp.current_user_saved_tracks(limit=50, offset=offset)
        
        if len(saved_tracks['items']) == 0:
            break
        
        artist_names = []
        artist_ids = []
        track_names = []
        track_ids = []

        for item in saved_tracks["items"]:
            artist_name = item["track"]["artists"][0]["name"]
            artist_id = item["track"]["artists"][0]["id"]
            track_name = item["track"]["name"]
            track_id = item["track"]["id"]

            artist_names.append(artist_name)
            artist_ids.append(artist_id)
            track_names.append(track_name)
            track_ids.append(track_id)

        saved_tracks_group.append(pd.DataFrame({
            "Artist Name": artist_names,
            "Artist ID": artist_ids,
            "Track Name": track_names,
            "Track ID": track_ids
        }))
        offset += 50

    saved_tracks_df = pd.concat(saved_tracks_group)

    return saved_tracks_df

In [4]:

def get_current_user_recently_played():
    recently_played = sp.current_user_recently_played(limit=50)

    artist_names = []
    artist_ids = []
    track_names = []
    track_ids = []

    for item in recently_played["items"]:
        artist_name = item["track"]["artists"][0]["name"]
        artist_id = item["track"]["artists"][0]["id"]
        track_name = item["track"]["name"]
        track_id = item["track"]["id"]
        
        artist_names.append(artist_name)
        artist_ids.append(artist_id)
        track_names.append(track_name)
        track_ids.append(track_id)

    recently_played_df = pd.DataFrame({
        "Artist Name": artist_names,
        "Artist ID": artist_ids,
        "Track Name": track_names,
        "Track ID": track_ids
    })

    return recently_played_df

In [5]:
def get_recommendation_genre_seeds():
    genres_dict = sp.recommendation_genre_seeds()
    genres_df = pd.DataFrame.from_dict(genres_dict)

    return genres_df

In [42]:
def get_currently_playing():
    currently_playing = sp.current_user_playing_track()

    if(currently_playing['context']):
        context_type = currently_playing['context']['type']
        uri = currently_playing['context']['uri']
    else:
        context_type = currently_playing['context']
        uri = currently_playing['context']

    track_name = currently_playing['item']['name']
    track_id = currently_playing['item']['id']
    album_name = currently_playing['item']['album']['name']
    album_id = currently_playing['item']['album']['id']

    artist_names = []
    artist_ids = []

    for artist in currently_playing['item']['artists']:
        artist_name = artist['name']
        artist_id = artist['id']
        
        artist_names.append(artist_name)
        artist_ids.append(artist_id)

    currently_playing_df = pd.DataFrame({
        'Context Type': context_type,
        'URI': uri,
        'Track Name': track_name,
        'Track ID': track_id,
        'Album Name': album_name,
        'Album ID': album_id,
        'Artist Names': [artist_names],
        'Artist IDs': [artist_ids]
        })

    return currently_playing_df